In [42]:
sc

In [43]:
from pyspark.sql.functions import col
import pyspark.sql.functions as sf
df = spark.read.load('joins_in_2015.csv', format='csv', header=True, inferSchema=False)
df.cache()

DataFrame[BOROUGH: string, CONTRIBUTING_FACTOR_VEHICLE_1: string, CONTRIBUTING_FACTOR_VEHICLE_2: string, CONTRIBUTING_FACTOR_VEHICLE_3: string, CONTRIBUTING_FACTOR_VEHICLE_4: string, CONTRIBUTING_FACTOR_VEHICLE_5: string, CROSS_STREET_NAME: string, DATETIME: string, LATITUDE: string, LOCATION: string, LONGITUDE: string, NUMBER_OF_CYCLIST_INJURED: string, NUMBER_OF_CYCLIST_KILLED: string, NUMBER_OF_MOTORIST_INJURED: string, NUMBER_OF_MOTORIST_KILLED: string, NUMBER_OF_PEDESTRIANS_INJURED: string, NUMBER_OF_PEDESTRIANS_KILLED: string, NUMBER_OF_PERSONS_INJURED: string, NUMBER_OF_PERSONS_KILLED: string, OFF_STREET_NAME: string, ON_STREET_NAME: string, UNIQUE_KEY: string, VEHICLE_TYPE_CODE_1: string, VEHICLE_TYPE_CODE_2: string, VEHICLE_TYPE_CODE_3: string, VEHICLE_TYPE_CODE_4: string, VEHICLE_TYPE_CODE_5: string, ZIP_CODE: string, _c28: string, Conditions: string, DateUTC: string, DatetimeEDT: string, Dew_PointF: string, Events: string, Freezing: string, Gust_SpeedMPH: string, Humidity: s

In [32]:
overall_df = df.filter(col('LATITUDE').isNotNull() & ~col('LATITUDE').like('0'))\
        .select("LOCATION", 'LATITUDE', 'LONGITUDE').groupBy("LOCATION", 'LATITUDE', 'LONGITUDE').count().orderBy('count', ascending=False)

overall_df.cache()    
    
# rain_df = df.filter(col('Rain').like("1") & col('LATITUDE').isNotNull() & ~col('LATITUDE').like('0'))\
#     .select("LOCATION", 'LATITUDE', 'LONGITUDE').groupBy("LOCATION", 'LATITUDE', 'LONGITUDE').count().orderBy('count', ascending=False)

# snow_df = df.filter(col('Snow').like("1") & col('LATITUDE').isNotNull() & ~col('LATITUDE').like('0'))\
#     .select("LOCATION", 'LATITUDE', 'LONGITUDE').groupBy("LOCATION", 'LATITUDE', 'LONGITUDE').count().orderBy('count', ascending=False)

# windy_df = df.filter(col('Windy').like("1") & col('LATITUDE').isNotNull() & ~col('LATITUDE').like('0'))\
#     .select("LOCATION", 'LATITUDE', 'LONGITUDE').groupBy("LOCATION", 'LATITUDE', 'LONGITUDE').count().orderBy('count', ascending=False)

# low_visibility_df = df.filter(col('LowVisibility').like("1") & col('LATITUDE').isNotNull() & ~col('LATITUDE').like('0'))\
#     .select("LOCATION", 'LATITUDE', 'LONGITUDE').groupBy("LOCATION", 'LATITUDE', 'LONGITUDE').count().orderBy('count', ascending=False)

# inclement_df = df.filter((col('Rain').like("1") | col('Snow').like("1") | col('Windy').like("1") | col('LowVisibility').like("1") | \
#     col('Freezing').like("1")) & col('LATITUDE').isNotNull() & ~col('LATITUDE').like('0')).select("LOCATION", 'LATITUDE', 'LONGITUDE')\
#     .groupBy("LOCATION", 'LATITUDE', 'LONGITUDE').count().orderBy('count', ascending=False)

# freezing_df = df.filter(col('Freezing').like("1") & col('LATITUDE').isNotNull() & ~col('LATITUDE').like('0'))\
#      .select("LOCATION", 'LATITUDE', 'LONGITUDE').groupBy("LOCATION", 'LATITUDE', 'LONGITUDE').count().orderBy('count', ascending=False)


DataFrame[LOCATION: string, LATITUDE: string, LONGITUDE: string, count: bigint]

In [44]:
# overall_df.show()

In [48]:
rain_counts = df.select("LOCATION", 'LATITUDE', 'LONGITUDE', df.Rain.cast("integer").alias("Rain"))\
.groupBy("LOCATION", 'LATITUDE', 'LONGITUDE').agg(sf.sum('Rain').alias('rain_count')).orderBy('rain_count', ascending=False)

rain_counts.show()

+--------------------+----------+-----------+----------+
|            LOCATION|  LATITUDE|  LONGITUDE|rain_count|
+--------------------+----------+-----------+----------+
|                null|      null|       null|      1815|
|(40.8100528, -73....|40.8100528|-73.9251435|        11|
|(40.8285193, -73....|40.8285193|-73.8411794|        11|
|(40.6685062, -73....|40.6685062|-73.9256052|         9|
|(40.6694023, -73....|40.6694023|-73.9421971|         9|
|(40.759308, -73.9...| 40.759308|-73.9652931|         9|
|(40.7596199, -73....|40.7596199|-73.9619648|         8|
|(40.7850321, -73....|40.7850321|-73.9405708|         7|
|(40.6482565, -73....|40.6482565|-73.9713978|         7|
|(40.6774132, -73....|40.6774132|-73.9303181|         7|
|(40.820189, -73.8...| 40.820189|-73.8906752|         7|
|(40.7139131, -73....|40.7139131|-73.8300825|         7|
|(40.7197624, -73....|40.7197624|-73.9447576|         7|
|(40.8289603, -73....|40.8289603|-73.8420547|         7|
|(40.7584292, -73....|40.758429

In [49]:
rain_counts.count()

35408

In [50]:
joined_rain_counts = overall_df.join(rain_counts.select("LOCATION","rain_count"), "LOCATION")
joined_rain_counts.show()

+--------------------+----------+-----------+-----+----------+
|            LOCATION|  LATITUDE|  LONGITUDE|count|rain_count|
+--------------------+----------+-----------+-----+----------+
|(40.8100528, -73....|40.8100528|-73.9251435|  219|        11|
|(40.6528929, -73....|40.6528929|-73.8667311|  136|         4|
|(40.6960346, -73....|40.6960346|-73.9845292|  135|         6|
|(40.7606005, -73....|40.7606005|-73.9643142|  132|         4|
|(40.7460781, -73....|40.7460781|-73.9749359|  116|         7|
|(40.7595273, -73....|40.7595273| -73.999242|  112|         6|
|(40.6757357, -73....|40.6757357|-73.8968533|  111|         3|
|(40.7572323, -73....|40.7572323|-73.9897922|  108|         1|
|(40.7203211, -73....|40.7203211|-73.9940403|  105|         6|
|(40.7602226, -73....|40.7602226| -73.967462|  103|         4|
|(40.6585778, -73....|40.6585778|-73.8906229|   96|         3|
|(40.7584292, -73....|40.7584292|-73.9926328|   95|         7|
|(40.8571285, -73....|40.8571285|-73.8807926|   92|    

In [52]:
rain_percents = joined_rain_counts.select("*", ((joined_rain_counts['rain_count']/joined_rain_counts['count'])*100).alias("percent_rain")).orderBy(['percent_rain','count'], ascending=False)
rain_percents.show()

+--------------------+----------+-----------+-----+----------+------------+
|            LOCATION|  LATITUDE|  LONGITUDE|count|rain_count|percent_rain|
+--------------------+----------+-----------+-----+----------+------------+
|(40.7710869, -73....|40.7710869| -73.804419|    3|         3|       100.0|
|(40.7076317, -73....|40.7076317| -73.915841|    3|         3|       100.0|
|(40.7318009, -73....|40.7318009|-73.7170754|    2|         2|       100.0|
|(40.8566278, -73....|40.8566278|-73.9393519|    2|         2|       100.0|
|(40.7365763, -73....|40.7365763|-73.9077101|    2|         2|       100.0|
|(40.7658552, -73....|40.7658552|-73.9145003|    2|         2|       100.0|
|(40.767701, -73.7...| 40.767701|-73.7792302|    2|         2|       100.0|
|(40.6993586, -73....|40.6993586|-73.8423382|    2|         2|       100.0|
|(40.5880534, -74....|40.5880534| -74.093535|    2|         2|       100.0|
|(40.77396, -73.91...|  40.77396| -73.917761|    2|         2|       100.0|
|(40.8610017

In [80]:
# test_percent = rain_percents.filter(col('percent_rain') > 50).filter(col('percent_rain') < 100)
test_percent = rain_percents.filter(col('percent_rain') >= 0)
test_percent.count()
# test_percent.show(42)

35407

In [57]:
rain_counts = df.select("LOCATION", 'LATITUDE', 'LONGITUDE', df.Rain.cast("integer").alias("Rain"))\
.groupBy("LOCATION", 'LATITUDE', 'LONGITUDE').agg(sf.sum('Rain').alias('rain_count')).orderBy('rain_count', ascending=False)

joined_rain_counts = overall_df.join(rain_counts.select("LOCATION","rain_count"), "LOCATION")

rain_percents = joined_rain_counts.select("*", ((joined_rain_counts['rain_count']/joined_rain_counts['count'])*100).alias("percent_rain")).orderBy(['percent_rain','count'], ascending=False)
rain_percents.show()

+--------------------+----------+-----------+-----+----------+------------+
|            LOCATION|  LATITUDE|  LONGITUDE|count|rain_count|percent_rain|
+--------------------+----------+-----------+-----+----------+------------+
|(40.7710869, -73....|40.7710869| -73.804419|    3|         3|       100.0|
|(40.7076317, -73....|40.7076317| -73.915841|    3|         3|       100.0|
|(40.7318009, -73....|40.7318009|-73.7170754|    2|         2|       100.0|
|(40.7365763, -73....|40.7365763|-73.9077101|    2|         2|       100.0|
|(40.6993586, -73....|40.6993586|-73.8423382|    2|         2|       100.0|
|(40.7381238, -73....|40.7381238|-73.7046388|    2|         2|       100.0|
|(40.5880534, -74....|40.5880534| -74.093535|    2|         2|       100.0|
|(40.682097, -73.8...| 40.682097| -73.801709|    2|         2|       100.0|
|(40.7200577, -74....|40.7200577|-74.0056566|    2|         2|       100.0|
|(40.7658552, -73....|40.7658552|-73.9145003|    2|         2|       100.0|
|(40.77396, 

In [93]:
rain_counts = df.select("LOCATION", 'LATITUDE', 'LONGITUDE', df.Rain.cast("integer").alias("Rain"))\
.groupBy("LOCATION", 'LATITUDE', 'LONGITUDE').agg(sf.sum('Rain').alias('rain_count')).orderBy('rain_count', ascending=False)

joined_rain_counts = overall_df.join(rain_counts.select("LOCATION","rain_count"), "LOCATION")

rain_percents = joined_rain_counts.select("*", ((joined_rain_counts['rain_count']/joined_rain_counts['count'])*100).alias("percent_rain")).orderBy(['percent_rain','count'], ascending=False)
rain_percents.show()

+--------------------+----------+-----------+-----+----------+------------+
|            LOCATION|  LATITUDE|  LONGITUDE|count|rain_count|percent_rain|
+--------------------+----------+-----------+-----+----------+------------+
|(40.7076317, -73....|40.7076317| -73.915841|    3|         3|       100.0|
|(40.7710869, -73....|40.7710869| -73.804419|    3|         3|       100.0|
|(40.7318009, -73....|40.7318009|-73.7170754|    2|         2|       100.0|
|(40.7365763, -73....|40.7365763|-73.9077101|    2|         2|       100.0|
|(40.6993586, -73....|40.6993586|-73.8423382|    2|         2|       100.0|
|(40.762987, -73.7...| 40.762987|-73.7285343|    2|         2|       100.0|
|(40.5880534, -74....|40.5880534| -74.093535|    2|         2|       100.0|
|(40.682097, -73.8...| 40.682097| -73.801709|    2|         2|       100.0|
|(40.7200577, -74....|40.7200577|-74.0056566|    2|         2|       100.0|
|(40.903558, -73.8...| 40.903558|-73.8441798|    2|         2|       100.0|
|(40.7381238

In [59]:
snow_counts = df.select("LOCATION", 'LATITUDE', 'LONGITUDE', df.Snow.cast("integer").alias("snow"))\
.groupBy("LOCATION", 'LATITUDE', 'LONGITUDE').agg(sf.sum('snow').alias('snow_count')).orderBy('snow_count', ascending=False)

joined_snow_counts = overall_df.join(snow_counts.select("LOCATION","snow_count"), "LOCATION")

snow_percents = joined_snow_counts.select("*", ((joined_snow_counts['snow_count']/joined_snow_counts['count'])*100).alias("percent_snow")).orderBy(['percent_snow','count'], ascending=False)
snow_percents.show()

+--------------------+----------+-----------+-----+----------+------------+
|            LOCATION|  LATITUDE|  LONGITUDE|count|snow_count|percent_snow|
+--------------------+----------+-----------+-----+----------+------------+
|(40.7117777, -73....|40.7117777|-73.9104143|    2|         2|       100.0|
|(40.5980988, -73....|40.5980988|-73.9434111|    2|         2|       100.0|
|(40.7717379, -73....|40.7717379|-73.8014927|    2|         2|       100.0|
|(40.7123496, -73....|40.7123496|-73.8443563|    2|         2|       100.0|
|(40.5960383, -74....|40.5960383| -74.082416|    2|         2|       100.0|
|(40.7406421, -73....|40.7406421|-73.7581875|    2|         2|       100.0|
|(40.7925339, -73....|40.7925339| -73.841695|    2|         2|       100.0|
|(40.7093579, -73....|40.7093579|-73.8080447|    2|         2|       100.0|
|(40.7161154, -73....|40.7161154| -73.824898|    2|         2|       100.0|
|(40.5875049, -73....|40.5875049|-73.9259402|    2|         2|       100.0|
|(40.6922182

In [61]:
windy_counts = df.select("LOCATION", 'LATITUDE', 'LONGITUDE', df.Windy.cast("integer").alias("windy"))\
.groupBy("LOCATION", 'LATITUDE', 'LONGITUDE').agg(sf.sum('windy').alias('windy_count')).orderBy('windy_count', ascending=False)

joined_windy_counts = overall_df.join(windy_counts.select("LOCATION","windy_count"), "LOCATION")

windy_percents = joined_windy_counts.select("*", ((joined_windy_counts['windy_count']/joined_windy_counts['count'])*100).alias("percent_windy")).orderBy(['percent_windy','count'], ascending=False)
windy_percents.show()

+--------------------+----------+-----------+-----+-----------+-------------+
|            LOCATION|  LATITUDE|  LONGITUDE|count|windy_count|percent_windy|
+--------------------+----------+-----------+-----+-----------+-------------+
|(40.5621305, -74....|40.5621305|-74.1334081|    2|          2|        100.0|
|(40.6849302, -73....|40.6849302| -73.882056|    1|          1|        100.0|
|(40.5653054, -74....|40.5653054|-74.1837023|    1|          1|        100.0|
|(40.6895098, -73....|40.6895098|-73.8044275|    1|          1|        100.0|
|(40.7598833, -73....|40.7598833| -73.878559|    1|          1|        100.0|
|(40.7029733, -73....|40.7029733|-73.7636692|    1|          1|        100.0|
|(40.8331857, -73....|40.8331857|-73.9302128|    1|          1|        100.0|
|(40.7176557, -73....|40.7176557|-73.7967923|    1|          1|        100.0|
|(40.7497466, -73....|40.7497466|-73.9699466|    1|          1|        100.0|
|(40.6408362, -74....|40.6408362|-74.1171202|    1|          1| 

In [62]:
low_visibility_counts = df.select("LOCATION", 'LATITUDE', 'LONGITUDE', df.LowVisibility.cast("integer").alias("low_visibility"))\
.groupBy("LOCATION", 'LATITUDE', 'LONGITUDE').agg(sf.sum('low_visibility').alias('low_visibility_count')).orderBy('low_visibility_count', ascending=False)

joined_low_visibility_counts = overall_df.join(low_visibility_counts.select("LOCATION","low_visibility_count"), "LOCATION")

low_visibility_percents = joined_low_visibility_counts.select("*", ((joined_low_visibility_counts['low_visibility_count']/joined_low_visibility_counts['count'])*100).alias("percent_low_visibility")).orderBy(['percent_low_visibility','count'], ascending=False)
low_visibility_percents.show()

+--------------------+----------+-----------+-----+--------------------+----------------------+
|            LOCATION|  LATITUDE|  LONGITUDE|count|low_visibility_count|percent_low_visibility|
+--------------------+----------+-----------+-----+--------------------+----------------------+
|(40.7076317, -73....|40.7076317| -73.915841|    3|                   3|                 100.0|
|(40.7710869, -73....|40.7710869| -73.804419|    3|                   3|                 100.0|
|(40.7496779, -73....|40.7496779|-73.8267358|    2|                   2|                 100.0|
|(40.6874871, -73....|40.6874871|-73.7505666|    2|                   2|                 100.0|
|(40.8961082, -73....|40.8961082| -73.847742|    2|                   2|                 100.0|
|(40.7925339, -73....|40.7925339| -73.841695|    2|                   2|                 100.0|
|(40.6112857, -74....|40.6112857|-74.1087305|    2|                   2|                 100.0|
|(40.6273254, -73....|40.6273254|-73.992

In [63]:
freezing_counts = df.select("LOCATION", 'LATITUDE', 'LONGITUDE', df.Freezing.cast("integer").alias("freezing"))\
.groupBy("LOCATION", 'LATITUDE', 'LONGITUDE').agg(sf.sum('freezing').alias('freezing_count')).orderBy('freezing_count', ascending=False)

joined_freezing_counts = overall_df.join(freezing_counts.select("LOCATION","freezing_count"), "LOCATION")

freezing_percents = joined_freezing_counts.select("*", ((joined_freezing_counts['freezing_count']/joined_freezing_counts['count'])*100).alias("percent_freezing")).orderBy(['percent_freezing','count'], ascending=False)
freezing_percents.show()

+--------------------+----------+-----------+-----+--------------+----------------+
|            LOCATION|  LATITUDE|  LONGITUDE|count|freezing_count|percent_freezing|
+--------------------+----------+-----------+-----+--------------+----------------+
|(40.6649238, -73....|40.6649238|-73.8241022|    4|             4|           100.0|
|(40.6994527, -73....|40.6994527|-73.8221311|    4|             4|           100.0|
|(40.6661608, -73....|40.6661608|-73.8415001|    3|             3|           100.0|
|(40.6765112, -73....|40.6765112|-73.8326942|    3|             3|           100.0|
|(40.6900457, -73....|40.6900457|-73.8616591|    3|             3|           100.0|
|(40.829141, -73.9...| 40.829141|-73.9081027|    3|             3|           100.0|
|(40.6826929, -73....|40.6826929|-73.9904237|    3|             3|           100.0|
|(40.5962949, -74....|40.5962949|-74.0949004|    3|             3|           100.0|
|(40.6254635, -73....|40.6254635|-73.9838323|    3|             3|          

In [94]:
overall_join = overall_df.join(rain_percents.select("LOCATION","rain_count", "percent_rain"), "LOCATION")
overall_join = overall_join.join(snow_percents.select("LOCATION","snow_count", "percent_snow"), "LOCATION")
overall_join = overall_join.join(windy_percents.select("LOCATION","windy_count", "percent_windy"), "LOCATION")
overall_join = overall_join.join(low_visibility_percents.select("LOCATION","low_visibility_count", "percent_low_visibility"), "LOCATION")
overall_join = overall_join.join(freezing_percents.select("LOCATION","freezing_count", "percent_freezing"), "LOCATION")

overall_join.take(1)

[Row(LOCATION=u'(40.5255456, -74.2054595)', LATITUDE=u'40.5255456', LONGITUDE=u'-74.2054595', count=4, rain_count=1, percent_rain=25.0, snow_count=0, percent_snow=0.0, windy_count=0, percent_windy=0.0, low_visibility_count=1, percent_low_visibility=25.0, freezing_count=0, percent_freezing=0.0)]

In [95]:
overall_join.coalesce(1).write.csv("2015_coords_for_carto_normalized/2015_overall_coords_norm", header=True)

In [97]:
rain_percents_filtered = rain_percents.filter(col('rain_count') > 0)
snow_percents_filtered = snow_percents.filter(col('snow_count') > 0)
windy_percents_filtered = windy_percents.filter(col('windy_count') > 0)
low_visibility_percents_filtered = low_visibility_percents.filter(col('low_visibility_count') > 0)
freezing_percents_filtered = freezing_percents.filter(col('freezing_count') > 0)

In [102]:
freezing_percents_filtered.count()

12781

In [88]:
rain_percents_filtered.coalesce(1).write.csv("2015_coords_for_carto_normalized/2015_rain_coords_norm", header=True)
snow_percents_filtered.coalesce(1).write.csv("2015_coords_for_carto_normalized/2015_snow_coords_norm", header=True)
windy_percents_filtered.coalesce(1).write.csv("2015_coords_for_carto_normalized/2015_windy_coords_norm", header=True)
low_visibility_percents_filtered.coalesce(1).write.csv("2015_coords_for_carto_normalized/2015_low_visibility_coords_norm", header=True)
freezing_percents_filtered.coalesce(1).write.csv("2015_coords_for_carto_normalized/2015_freezing_coords_norm", header=True)

In [90]:
overall_join.count()

35407

In [92]:
overall_join.take(1)

[Row(LOCATION=u'(40.5255456, -74.2054595)', LATITUDE=u'40.5255456', LONGITUDE=u'-74.2054595', count=4, rain_count=1, percent_rain=25.0, snow_count=0, percent_snow=0.0, snow_count=0, percent_snow=0.0, snow_count=0, percent_snow=0.0, windy_count=0, percent_windy=0.0, snow_count=0, percent_snow=0.0, windy_count=0, percent_windy=0.0, low_visibility_count=1, percent_low_visibility=25.0, freezing_count=0, percent_freezing=0.0)]